In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)




import os
UpbitFile=[]
BinanceFile=[]
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        
        if dirname =='/kaggle/input/crypto-signal/Upbit':
            UpbitFile.append(filename)
        elif dirname =='/kaggle/input/crypto-signal/Binance':
            BinanceFile.append(filename)


In [2]:
#!pip install qtpylib
!pip install talib-binary

     |████████████████████████████████| 2.4 MB 3.1 MB/s 


In [3]:
import talib.abstract as ta

# Load data
* upbit => Korean crypto market
* binance => Global crypto market

In [4]:


#select crypto_currencies for the models.
usdt_crypto_list=[
'ETH'
#'BTC',
#  'LTC',
# 'XRP',
# 'ETC',
#   'OMG',
#  'ADA',
#  'SC',
#  'TRX',
#  'BCH',
#  'DGB',
#  'DOGE',
#  'ZRX'
#  'RVN',
#  'BAT'
]

DIR_Binance = '/kaggle/input/crypto-signal/Binance/'
DIR_Upbit = '/kaggle/input/crypto-signal/Upbit/'
binanace_usdt_crpyto = [ele+"USDT" for ele in usdt_crypto_list]
upbit_usdt_crpyto = ["USDT-"+ele for ele in usdt_crypto_list]

# Match and set time index both the Upbit data and the Binance data

In [5]:

import datetime as dt
from datetime import datetime
for cryptoName in usdt_crypto_list:
    print(cryptoName)
    globals()[cryptoName+'_1_Binance'] = pd.concat([pd.read_csv(DIR_Binance+file) for file in BinanceFile if cryptoName.lower() in file.lower()])
    globals()[cryptoName+'_D_Upbit'] = pd.concat([pd.read_csv(DIR_Upbit+file) for file in UpbitFile if cryptoName.lower()+'_d' in file.lower()])
    globals()[cryptoName+'_1_Binance']['Open_time'] = globals()[cryptoName+'_1_Binance'].apply(lambda x: x['Open_time'][:19],axis=1)
    globals()[cryptoName+'_1_Binance']['time_index'] = globals()[cryptoName+'_1_Binance'].apply(lambda x: dt.datetime.strptime(x['Open_time'], '%Y-%m-%d %H:%M:%S'),axis=1)
    globals()[cryptoName+'_1_Binance'].set_index('time_index', inplace=True)
    globals()[cryptoName+'_1_Binance'] = globals()[cryptoName+'_1_Binance'].sort_index()
    globals()[cryptoName+'_1_Binance'] = globals()[cryptoName+'_1_Binance'].drop_duplicates()
    globals()[cryptoName+'_D_Upbit'] = globals()[cryptoName+'_D_Upbit'].drop_duplicates()

ETH


# 

# Functions

In [6]:
def BB(df ,Col_name , BB_len=20, unit=2,isFutureVal=1):
    if len(df)<BB_len:
        print("check_range")
        return
    df=df.copy()
    rolling = df[Col_name].rolling(BB_len)
    if isFutureVal:
        df[Col_name+'Mean_'+str(BB_len)] = rolling.mean().shift(1)
        df[Col_name+'unit_STD_'+str(BB_len)]=rolling.std().shift(1)*unit
        df[Col_name+'Min_'+str(BB_len)] = rolling.min().shift(1)
        df[Col_name+'Max_'+str(BB_len)]= rolling.max().shift(1)
    else:
        df[Col_name+'Mean_'+str(BB_len)] = rolling.mean()
        df[Col_name+'unit_STD_'+str(BB_len)]=rolling.std()*unit
        df[Col_name+'Min_'+str(BB_len)] = rolling.min()
        df[Col_name+'Max_'+str(BB_len)]= rolling.max()
    return df



def RSI(df,RSI_len=14,isFutureVal=1):
    if len(df)<RSI_len:
        print("check_range")
        return
    df = df.copy()
    if "upNdown" not in df.columns:
        df['Close_1_sft']=df['Close'].shift(1)
        df['upNdown']= df.apply(lambda x: 0 if np.isnan(x['Close']-x['Close_1_sft']) else x['Close']-x['Close_1_sft'] ,axis=1)
        df["UP"] = df["upNdown"].apply(lambda x: x if x > 0 else 0)
        df["DOWN"] = df["upNdown"].apply(lambda x: x*(-1) if x < 0 else 0)
    if isFutureVal:
        rolling_upNdown = df[["UP","DOWN"]].rolling(RSI_len).mean()
        df["RSI_avgUp_"+str(RSI_len)] =  rolling_upNdown["UP"].shift(1).replace(0 ,np.nan)
        df["RSI_avgDown_"+str(RSI_len)] = rolling_upNdown["DOWN"].shift(1).replace(0 ,np.nan)
        df["RSI_"+str(RSI_len)] = df.apply(lambda x: x["RSI_avgUp_"+str(RSI_len)]/(x["RSI_avgUp_"+str(RSI_len)]+x["RSI_avgDown_"+str(RSI_len)])*100,axis=1 )
    else:
        rolling_upNdown = df[["UP","DOWN"]].rolling(RSI_len).mean()
        df["RSI_avgUp_"+str(RSI_len)] =  rolling_upNdown["UP"].replace(0 ,np.nan)
        df["RSI_avgDown_"+str(RSI_len)] = rolling_upNdown["DOWN"].replace(0 ,np.nan)
        df["RSI_"+str(RSI_len)] = df.apply(lambda x: x["RSI_avgUp_"+str(RSI_len)]/(x["RSI_avgUp_"+str(RSI_len)]+x["RSI_avgDown_"+str(RSI_len)])*100,axis=1 )
    
    del df["RSI_avgUp_"+str(RSI_len)],df["RSI_avgDown_"+str(RSI_len)]

    return df

def MFI(df,MFI_len=14,isFutureVal=1):
    if len(df)<MFI_len:
        print("check_range")
        return
    
    df = df.copy()
    if "upNdown" not in df.columns:
        df['Close_1_sft']=df['Close'].shift(1)
        df['upNdown']= df.apply(lambda x: 0 if np.isnan(x['Close']-x['Close_1_sft']) else x['Close']-x['Close_1_sft'] ,axis=1)
        df["UP"] = df["upNdown"].apply(lambda x: x if x > 0 else 0)
        df["DOWN"] = df["upNdown"].apply(lambda x: x*(-1) if x < 0 else 0)

    if "MoneyFlow" not in df.columns:
        df['HighLowClose'] = df[['Close','High','Low']].mean(axis=1)
        df['MoneyFlow'] = df['HighLowClose']*df['Volume']
        
    df['MFI_UP'] = df['MoneyFlow']*df['UP']
    df['MFI_DOWN'] = df['MoneyFlow']*df['DOWN']
    if isFutureVal:
        rolling_upNdown = df[["MFI_UP","MFI_DOWN"]].rolling(MFI_len).mean().shift(1)
    else:    
        rolling_upNdown = df[["MFI_UP","MFI_DOWN"]].rolling(MFI_len).mean()
        
    df["MFI_avgUp_"+str(MFI_len)] =  rolling_upNdown["MFI_UP"]
    df["MFI_avgDown_"+str(MFI_len)] = rolling_upNdown["MFI_DOWN"]
    df["MFI_avgUp_"+str(MFI_len)]=df["MFI_avgUp_"+str(MFI_len)].replace(0 ,np.nan)
    df["MFI_avgDown_"+str(MFI_len)]=df["MFI_avgDown_"+str(MFI_len)].replace(0 ,np.nan)
    df["MFI_"+str(MFI_len)] = df.apply(lambda x: x["MFI_avgUp_"+str(MFI_len)]/(x["MFI_avgUp_"+str(MFI_len)]+x["MFI_avgDown_"+str(MFI_len)])*100,axis=1 )
    
    del df['MFI_UP'], df['MFI_DOWN'],df["MFI_avgUp_"+str(MFI_len)],df["MFI_avgDown_"+str(MFI_len)],df["UP"],df["DOWN"],df["upNdown"],df['MoneyFlow'],df['HighLowClose'],df['Close_1_sft']

    return df

def MACD(df, MACD_short=12,MACD_long=26,MACD_signal=9,isFutureVal=1):
    if len(df)<MACD_long:
        print("check_range")
        return
    
    df = df.copy()
    if isFutureVal:
        df["MACD_short"]=df["Close"].ewm(span=MACD_short).mean().shift(1)
        df["MACD_long"]=df["Close"].ewm(span=MACD_long).mean().shift(1)
    else:
        df["MACD_short"]=df["Close"].ewm(span=MACD_short).mean()
        df["MACD_long"]=df["Close"].ewm(span=MACD_long).mean()
        
    df["MACD"]=df.apply(lambda x: (x["MACD_short"]-x["MACD_long"]), axis=1) 
    df["MACD_signal"]=df["MACD"].ewm(span=MACD_signal).mean() 
    del df["MACD_short"],df["MACD_long"]
    return df


def Stochastic(df, option = 1,isFutureVal=1):
    sto_N=5
    sto_m=3
    sto_t=3


    if option ==2:

        sto_N=14
        sto_m=1
        sto_t=3
    elif option ==3:
        sto_N=5
        sto_m=3
        sto_t=3
    elif option ==4:
        sto_N=10
        sto_m=6
        sto_t=6
    elif option ==5:
        sto_N=20
        sto_m=12
        sto_t=12
    df = df.copy()

    if "max_%d"%sto_N not in df.columns:
        df["max_%d"%sto_N]=df["High"].rolling(sto_N).max()
    if  "min_%d"%sto_N not in df.columns:
        df["min_%d"%sto_N]=df["Low"].rolling(sto_N).min()
    df["stochasticK_%d"%option]=df.apply(lambda x:100*(x["Close"]-x["min_%d"%sto_N])/
                        (x["max_%d"%sto_N]-x["min_%d"%sto_N]) 
                                        if (x["max_%d"%sto_N]-x["min_%d"%sto_N])!=0 else 50,1)
    if isFutureVal:
        df["slow_K_%d"%option]=df["stochasticK_%d"%option].rolling(sto_m).mean().shift(1)
        df["slow_D_%d"%option]=df["slow_K_%d"%option].rolling(sto_t).mean().shift(1)
    else:
        df["slow_K_%d"%option]=df["stochasticK_%d"%option].rolling(sto_m).mean()
        df["slow_D_%d"%option]=df["slow_K_%d"%option].rolling(sto_t).mean()
        
    del df["stochasticK_%d"%option]
    return df



def Slope(df,shiftNum,shiftCols):
    if len(df)<abs(shiftNum):
        print("check_range")
        return

    df = df.copy()

    for col in shiftCols:
        df[col+'_slope_%d'%shiftNum] = df[col]/df[col].shift(shiftNum)
        
    return df


def ShiftNadd(df,col_names,shift_num):
    df = df.copy()
    df[[col+'_shift_'+str(shift_num) for col in col_names]] = df[col_names].shift(shift_num)
    return df

# https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type == 'datetime64[ns]':
            continue
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

def TrainData_startTimeSetting(splitted_index_setTime, set_year=2021, set_month=1, set_day=1, set_hr=0, set_min=0):
    set_time = datetime(int(set_year),int(set_month),int(set_day),int(set_hr),int(set_min))
    if set_time<splitted_index_setTime[0][0]:
        print('no data// change the setting time')
        return
    result = []
    for idx,period in enumerate(splitted_index_setTime):
        
        if period[0]<set_time<period[1]:
            result.append((set_time,period[1]))
            result = result + splitted_index_setTime[idx+1:]
            return result
        else:
            continue
            
def trainTestConvert(train_df,feature):
    target_num = [310,315,320,3,510,515,520,5,7,12,15]
    
    target_num.remove(feature)
    drop_col = ['MaxFuture_'+str(x) for x in target_num]
    train_df = train_df.drop(columns=['Open_time', 'quote_av',	'trades',	'tb_base_av',	'tb_quote_av',	'Symbol',	'DAY_UTC_Day'],)
    train_df = train_df.drop(columns=drop_col)
    train_df = train_df.drop(train_df[train_df['MaxFuture_'+str(feature)].isnull()].index)
    train_df.rename(columns={'MaxFuture_'+str(feature): 'target'}, inplace=True)
    columns_name = train_df.columns
    train_x = train_df[columns_name[0:-1]]
    train_y = train_df[columns_name[-1]]
    
    return train_x,train_y


def futurePrice(df,lenTick=3):
    df = df.copy()
    df = df[::-1]
    df['MaxFuture_'+str(lenTick)] = df['High'].rolling(lenTick).max()
    df['MaxFuture_'+str(lenTick)] = df['MaxFuture_'+str(lenTick)].shift(1)
    df['MaxFuture_'+str(lenTick)] = df['MaxFuture_'+str(lenTick)]/df['Close']*100-100
    df = df[::-1]
    return df

from lightgbm import LGBMClassifier
from sklearn import metrics

def lgb_f1_score(y_hat, y_valid):
    y_hat = np.where(y_hat < 0.5, 0, 1) # scikits f1 doesn't like probabilities
    return 'f1', metrics.f1_score(y_hat,y_valid), True


def fit_lgbm(x_train, y_train, x_valid, y_valid, params: dict=None, verbose=200):
    oof_pred = np.zeros(len(y_valid), dtype=np.float32)
    model = LGBMClassifier(**params)
    evals_result = {}
    model.fit(x_train, y_train, 
              eval_set=[(x_valid, y_valid)],
              early_stopping_rounds = 100,
              
)
    oof_pred = model.predict(x_valid)
    score = metrics.roc_auc_score(oof_pred, y_valid)
    print('auc:', score)
    return model, score



# Calculate the crypto price difference between the Binance and the Upbit(korean)

In [7]:
#calculate price difference between Binance(American) and upbit(korean)

for cryptoName in usdt_crypto_list:
    print(cryptoName)
    
    #drop duplicates
    globals()[cryptoName+'_1_Binance'] = globals()[cryptoName+'_1_Binance'].drop_duplicates()
    globals()[cryptoName+'_D_Upbit'] = globals()[cryptoName+'_D_Upbit'].drop_duplicates()
    
    ##############################################################################
    #merge upbit-day open_price with binance open_price to get a price difference#
    ##############################################################################
    #set a public key(UTC,Open_time)
    globals()[cryptoName+'_D_Upbit']['UTC'] = globals()[cryptoName+'_D_Upbit'].apply(lambda x : x['UTC'][:10]+' '+x['UTC'][11:19], axis = 1 )
    globals()[cryptoName+'_1_Binance']['Open_time'] = globals()[cryptoName+'_1_Binance'].apply(lambda x : x['Open_time'][:19], axis = 1 )
    #merge two df
    globals()[cryptoName+'_D_Upbit'] = pd.merge(globals()[cryptoName+'_1_Binance'][['Open_time','Open']],globals()[cryptoName+'_D_Upbit'], left_on='Open_time',right_on='UTC')
    #calculate price difference between korean crypto price and american cryto price
    globals()[cryptoName+'_D_Upbit']['Kimchi_Pri'] = globals()[cryptoName+'_D_Upbit'].apply(lambda x : (x['opening_price']/(x['Open']))*100 if ~np.isnan(x['Open']) else np.nan,axis=1)
    globals()[cryptoName+'_D_Upbit']['UTC_Day'] = globals()[cryptoName+'_D_Upbit'].apply(lambda x : x['UTC'][:10],axis=1)
    globals()[cryptoName+'_D_Upbit']['Kimchi_Pri']=round(globals()[cryptoName+'_D_Upbit']['Kimchi_Pri'])

    #change column names
    globals()[cryptoName+'_D_Upbit'].rename(columns={'high_price': 'High', 'low_price': 'Low', 'trade_price': 'Close'}, inplace=True)
    globals()[cryptoName+'_D_Upbit'].rename(columns={'acc_trade_volume': 'Volume'}, inplace=True)
    
    #reduce memories
    globals()[cryptoName+'_D_Upbit'] = reduce_mem_usage(globals()[cryptoName+'_D_Upbit'])

ETH
Memory usage of dataframe is 0.15 MB
Memory usage after optimization is: 0.22 MB
Decreased by -51.1%


# Calculate statistical information and put them into each column 

In [8]:

DAY_Slope_1=['opening_price', 'Volume',
       'Kimchi_Pri', 'Kimchi_PriMean_5', 'Kimchi_Priunit_STD_5', 'Kimchi_Priunit_STD_20',
       'VolumeMean_5', 'Volumeunit_STD_5', 
       'VolumeMean_20', 'Volumeunit_STD_20', 
        'CloseMean_5', 'Closeunit_STD_5',
        'CloseMean_20', 'Closeunit_STD_20',
        'MFI_14', 'MACD','RSI_14',
       'slow_K_1', 'slow_D_1']
DAY_Slope_3_5=[
       'Kimchi_PriMean_5', 'Kimchi_Priunit_STD_5','Kimchi_PriMean_20', 'Kimchi_Priunit_STD_20',
       'VolumeMean_5', 
       'VolumeMean_20',
        'CloseMean_5', 
     'CloseMean_20', 
         'RSI_14',
        'MFI_14']
for cryptoName in usdt_crypto_list:
    print(cryptoName)
    globals()[cryptoName+'_D_Upbit'] = BB(globals()[cryptoName+'_D_Upbit'] ,'Kimchi_Pri' , BB_len=5, unit=2,isFutureVal=0)
    globals()[cryptoName+'_D_Upbit'] = BB(globals()[cryptoName+'_D_Upbit'] ,'Kimchi_Pri' , BB_len=20, unit=2,isFutureVal=0)
    globals()[cryptoName+'_D_Upbit'] = BB(globals()[cryptoName+'_D_Upbit'] ,'Volume' , BB_len=5, unit=2)
    globals()[cryptoName+'_D_Upbit'] = BB(globals()[cryptoName+'_D_Upbit'] ,'Volume' , BB_len=20, unit=2)
    globals()[cryptoName+'_D_Upbit'] = BB(globals()[cryptoName+'_D_Upbit'] ,'Volume' , BB_len=60, unit=2)
    globals()[cryptoName+'_D_Upbit'] = BB(globals()[cryptoName+'_D_Upbit'] ,'Volume' , BB_len=120, unit=2)
    globals()[cryptoName+'_D_Upbit'] = BB(globals()[cryptoName+'_D_Upbit'] ,'Volume' , BB_len=200, unit=2)
    globals()[cryptoName+'_D_Upbit'] = BB(globals()[cryptoName+'_D_Upbit'] ,'Close' , BB_len=5, unit=2)
    globals()[cryptoName+'_D_Upbit'] = BB(globals()[cryptoName+'_D_Upbit'] ,'Close' , BB_len=20, unit=2)
    globals()[cryptoName+'_D_Upbit'] = BB(globals()[cryptoName+'_D_Upbit'] ,'Close' , BB_len=60, unit=2)
    globals()[cryptoName+'_D_Upbit'] = BB(globals()[cryptoName+'_D_Upbit'] ,'Close' , BB_len=120, unit=2)
    
    ########################################RSI ##################################################################
    #globals()[cryptoName+'_D_Upbit'] = RSI(globals()[cryptoName+'_D_Upbit'])
    globals()[cryptoName+'_D_Upbit']['RSI_14'] = ta.RSI(globals()[cryptoName+'_D_Upbit']['Close'],timeperiod = 14)
    globals()[cryptoName+'_D_Upbit'][['RSI_14']] = globals()[cryptoName+'_D_Upbit'][['RSI_14']].shift(1)
    ##########################################################################################################
    
    ##########################################################################################################
    #globals()[cryptoName+'_D_Upbit'] = MACD(globals()[cryptoName+'_D_Upbit'])
    macd,signal,_=ta.MACD(globals()[cryptoName+'_D_Upbit']['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    globals()[cryptoName+'_D_Upbit']['MACD'] = macd
    globals()[cryptoName+'_D_Upbit']['MACD_signal'] = signal
    globals()[cryptoName+'_D_Upbit']['MACD'] = globals()[cryptoName+'_D_Upbit']['MACD'].shift(1)
    globals()[cryptoName+'_D_Upbit']['MACD_signal'] = globals()[cryptoName+'_D_Upbit']['MACD_signal'].shift(1)
    ##########################################################################################################
    
    #globals()[cryptoName+'_D_Upbit'] = MFI(globals()[cryptoName+'_D_Upbit'])
    globals()[cryptoName+'_D_Upbit']['MFI_14'] = ta.MFI(globals()[cryptoName+'_D_Upbit']['High'], globals()[cryptoName+'_D_Upbit']['Low'], globals()[cryptoName+'_D_Upbit']['Close'], globals()[cryptoName+'_D_Upbit']['Volume'], timeperiod=14)
    globals()[cryptoName+'_D_Upbit']['MFI_14'] = globals()[cryptoName+'_D_Upbit']['MFI_14'].shift(1)
    ##############################################################################################################
    
    #globals()[cryptoName+'_D_Upbit'] = Stochastic(globals()[cryptoName+'_D_Upbit'])
    globals()[cryptoName+'_D_Upbit']['slow_K_1'], globals()[cryptoName+'_D_Upbit']['slow_D_1'] = ta.STOCH(globals()[cryptoName+'_D_Upbit']['High'], globals()[cryptoName+'_D_Upbit']['Low'], globals()[cryptoName+'_D_Upbit']['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    globals()[cryptoName+'_D_Upbit']['slow_K_1'] = globals()[cryptoName+'_D_Upbit']['slow_K_1'].shift(1)
    globals()[cryptoName+'_D_Upbit']['slow_D_1'] = globals()[cryptoName+'_D_Upbit']['slow_D_1'].shift(1)
    ################################################################################################################
    
    globals()[cryptoName+'_D_Upbit'] = Slope(globals()[cryptoName+'_D_Upbit'],1,DAY_Slope_1)
    globals()[cryptoName+'_D_Upbit'] = Slope(globals()[cryptoName+'_D_Upbit'],3,DAY_Slope_3_5)
    globals()[cryptoName+'_D_Upbit'] = Slope(globals()[cryptoName+'_D_Upbit'],5,DAY_Slope_3_5)
    globals()[cryptoName+'_D_Upbit'] = globals()[cryptoName+'_D_Upbit'].replace(np.inf, np.nan)
    globals()[cryptoName+'_D_Upbit'] = ShiftNadd(globals()[cryptoName+'_D_Upbit'],[ele+'_slope_1' for ele in DAY_Slope_1],1)
    globals()[cryptoName+'_D_Upbit'] = ShiftNadd(globals()[cryptoName+'_D_Upbit'],[ele+'_slope_1' for ele in DAY_Slope_1],2)
    #put 'DAY' for all columns to distinguish Day data and min data
    

    globals()[cryptoName+'_D_Upbit'].columns = ['DAY_'+ele for ele in globals()[cryptoName+'_D_Upbit'].columns]
    globals()[cryptoName+'_D_Upbit'] = globals()[cryptoName+'_D_Upbit'].drop(columns=['DAY_High', 'DAY_Low', 'DAY_Close', 'DAY_acc_trade_price','DAY_Open_time', 'DAY_Open', 'DAY_KST', 'DAY_UTC','DAY_Volume'])

ETH


In [9]:
globals()[cryptoName+'_D_Upbit']

,DAY_opening_price,DAY_Kimchi_Pri,DAY_UTC_Day,DAY_Kimchi_PriMean_5,DAY_Kimchi_Priunit_STD_5,DAY_Kimchi_PriMin_5,DAY_Kimchi_PriMax_5,DAY_Kimchi_PriMean_20,DAY_Kimchi_Priunit_STD_20,DAY_Kimchi_PriMin_20,...,DAY_Volumeunit_STD_20_slope_1_shift_2,DAY_CloseMean_5_slope_1_shift_2,DAY_Closeunit_STD_5_slope_1_shift_2,DAY_CloseMean_20_slope_1_shift_2,DAY_Closeunit_STD_20_slope_1_shift_2,DAY_MFI_14_slope_1_shift_2,DAY_MACD_slope_1_shift_2,DAY_RSI_14_slope_1_shift_2,DAY_slow_K_1_slope_1_shift_2,DAY_slow_D_1_slope_1_shift_2
0,298.00,99.0,2017-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,290.00,99.0,2017-08-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,294.50,101.0,2017-08-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,299.00,100.0,2017-08-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,322.75,100.0,2017-08-22,99.8,1.673320,99.0,101.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,3896.00,101.0,2021-10-16,100.2,2.190890,99.0,101.0,100.05,1.651156,98.0,...,0.986179,1.001587,1.377922,1.006923,1.027044,1.055310,1.071198,1.059916,1.143810,0.909734
1480,3826.00,100.0,2021-10-17,100.4,1.788854,99.0,101.0,100.05,1.651156,98.0,...,0.984842,1.012567,2.435736,1.012055,1.015197,1.016395,1.160019,1.072365,1.307555,1.109720
1481,3884.00,101.0,2021-10-18,100.4,1.788854,99.0,101.0,100.10,1.704483,98.0,...,1.026392,1.022250,1.555406,1.014246,1.031432,1.022029,1.202458,1.059346,1.359483,1.283902
1482,3778.00,101.0,2021-10-19,100.4,1.788854,99.0,101.0,100.15,1.750188,98.0,...,1.000352,1.018593,1.002515,1.012216,1.018079,1.010464,1.100032,0.973603,1.088162,1.228710


In [10]:
#set index for 1min binance data and sort by index

for cryptoName in usdt_crypto_list:
    print(cryptoName)  
    #reduce memory
    globals()[cryptoName+'_1_Binance'] = reduce_mem_usage(globals()[cryptoName+'_1_Binance'])

ETH
Memory usage of dataframe is 200.31 MB
Memory usage after optimization is: 158.39 MB
Decreased by 20.9%


# Find the non-sequential index

In [11]:

for cryptoName in usdt_crypto_list:
    print(cryptoName)
    start_year=globals()[cryptoName+'_1_Binance'].iloc[0][0][:4]
    start_month=globals()[cryptoName+'_1_Binance'].iloc[0][0][5:7]
    start_day=globals()[cryptoName+'_1_Binance'].iloc[0][0][8:10]
    start_hr=globals()[cryptoName+'_1_Binance'].iloc[0][0][11:13]
    start_min=globals()[cryptoName+'_1_Binance'].iloc[0][0][14:16]
    globals()[cryptoName+'_1_Binance_split_index'] = []
    start=datetime(int(start_year),int(start_month),int(start_day),int(start_hr),int(start_min))
    start_idx = 0
    for idx,time_stp in enumerate(globals()[cryptoName+'_1_Binance'].index):
        if time_stp!=(start+dt.timedelta(minutes=(idx - start_idx) )):
            globals()[cryptoName+'_1_Binance_split_index'].append((start,start+dt.timedelta(minutes=(idx - start_idx-1))))
            start =  time_stp
            start_idx = idx
    globals()[cryptoName+'_1_Binance_split_index'].append((start,start+dt.timedelta(minutes=((len(globals()[cryptoName+'_1_Binance'])-1) - start_idx-1))))

ETH


In [12]:
globals()[cryptoName+'_1_Binance']

,Open_time,Open,High,Low,Close,Volume,quote_av,trades,tb_base_av,tb_quote_av,Symbol
time_index,,,,,,,,,,,
2017-08-17 04:00:00,2017-08-17 04:00:00,301.25,301.25,301.25,301.25,0.426514,128.410873,2,0.426514,128.410873,ETHUSDT
2017-08-17 04:01:00,2017-08-17 04:01:00,301.25,301.25,301.25,301.25,2.757812,830.477417,4,2.757812,830.477417,ETHUSDT
2017-08-17 04:02:00,2017-08-17 04:02:00,300.00,300.00,300.00,300.00,0.099304,29.790001,2,0.099304,29.790001,ETHUSDT
2017-08-17 04:03:00,2017-08-17 04:03:00,300.00,300.00,300.00,300.00,0.313965,94.167000,3,0.000000,0.000000,ETHUSDT
2017-08-17 04:04:00,2017-08-17 04:04:00,301.25,301.25,301.25,301.25,0.232056,69.868179,1,0.232056,69.868179,ETHUSDT
...,...,...,...,...,...,...,...,...,...,...,...
2021-10-20 10:15:00,2021-10-20 10:15:00,3862.00,3864.00,3862.00,3862.00,65.812500,254308.906250,215,45.625000,176183.062500,ETHUSDT
2021-10-20 10:16:00,2021-10-20 10:16:00,3862.00,3864.00,3862.00,3864.00,46.906250,181261.468750,172,16.687500,64499.035156,ETHUSDT
2021-10-20 10:17:00,2021-10-20 10:17:00,3864.00,3864.00,3862.00,3862.00,113.687500,439087.250000,252,53.250000,205663.687500,ETHUSDT


# Make a merged dataFrame(Day_upbit + Min_Binance) 
# Set a strat-time for the dataFrame

In [13]:
# merge 
for cryptoName in usdt_crypto_list:
    
    ####################################################################################
    #Make a merged dataFrame(Day_upbit + Min_Binance) Set a strat-time for the dataFrame#
    ####################################################################################
    year= 2021
    month = 7
    day=1
    splits=TrainData_startTimeSetting(globals()[cryptoName+'_1_Binance_split_index'],year,month,day,0,0)
    
    start_train_time = datetime(int(year),int(month),int(day))
    
    globals()[cryptoName+'_1_Binance_'+str(year)+'_'+str(month)+'_df_arr']=[]
    for split in splits:
        if(len(globals()[cryptoName+'_1_Binance'][split[0]:split[1]])<600):
            continue
        df_tmp = globals()[cryptoName+'_1_Binance'][split[0]:split[1]].copy()
        df_tmp['DAY_UTC_Day'] = df_tmp.apply(lambda x : x['Open_time'][:10],axis=1)
        df_tmp = df_tmp.merge(globals()[cryptoName+'_D_Upbit'],on='DAY_UTC_Day')
        try:
            df_tmp['time_index'] = df_tmp.apply(lambda x: dt.datetime.strptime(x['Open_time'], '%Y-%m-%d %H:%M:%S'),axis=1)
            df_tmp['Min_delta']=df_tmp.apply(lambda x : (x['time_index'] - start_train_time).total_seconds()/60,axis=1)
            df_tmp.set_index('time_index', inplace=True)

        except:
            continue

        globals()[cryptoName+'_1_Binance_'+str(year)+'_'+str(month)+'_df_arr'].append(df_tmp)

In [14]:
globals()[cryptoName+'_1_Binance_'+str(year)+'_'+str(month)+'_df_arr'][0].columns[0:20]

Index(['Open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 'quote_av',
       'trades', 'tb_base_av', 'tb_quote_av', 'Symbol', 'DAY_UTC_Day',
       'DAY_opening_price', 'DAY_Kimchi_Pri', 'DAY_Kimchi_PriMean_5',
       'DAY_Kimchi_Priunit_STD_5', 'DAY_Kimchi_PriMin_5',
       'DAY_Kimchi_PriMax_5', 'DAY_Kimchi_PriMean_20',
       'DAY_Kimchi_Priunit_STD_20'],
      dtype='object')

# Put additional statistical information to the dataframe 

In [15]:
Slope_1_min=['Open', 
       'VolumeMean_5', 'Volumeunit_STD_5', 
       'VolumeMean_20', 'Volumeunit_STD_20', 
        'CloseMean_5', 'Closeunit_STD_5',
        'CloseMean_20', 'Closeunit_STD_20',
        'CloseMean_60', 'Closeunit_STD_60',
        'CloseMean_120', 
        'CloseMean_200', 'MACD','RSI_14',
       'slow_K_1', 'slow_D_1',
            ]
Slope_3_5_min=[
       'VolumeMean_5', 
       'VolumeMean_20',
        'CloseMean_5', 
         'CloseMean_20', 
        'CloseMean_60',
        'CloseMean_120',
        'CloseMean_200', 
         'MACD' ,'RSI_14']
max_min = ['CloseMax_20','CloseMax_60',
          'CloseMin_20','CloseMin_60','CloseMin_200'
          ]
for cryptoName in usdt_crypto_list:
    df_arr_v1=[]
    for idx, df in enumerate(globals()[cryptoName+'_1_Binance_'+str(year)+'_'+str(month)+'_df_arr']):
        print(idx/len(globals()[cryptoName+'_1_Binance_'+str(year)+'_'+str(month)+'_df_arr']))
        df = BB(df ,'Volume' , BB_len=5, unit=2,isFutureVal=0)
        df = BB(df ,'Volume' , BB_len=20, unit=2,isFutureVal=0)
        df = BB(df ,'Volume' , BB_len=60, unit=2,isFutureVal=0)
        df = BB(df ,'Volume' , BB_len=120, unit=2,isFutureVal=0)
        df = BB(df ,'Volume' , BB_len=200, unit=2,isFutureVal=0)
        df = BB(df ,'Close' , BB_len=5, unit=2,isFutureVal=0)
        df = BB(df ,'Close' , BB_len=20, unit=2,isFutureVal=0)
        df = BB(df ,'Close' , BB_len=60, unit=2,isFutureVal=0)
        df = BB(df ,'Close' , BB_len=120, unit=2,isFutureVal=0)
        df = BB(df ,'Close' , BB_len=200, unit=2,isFutureVal=0)

        
        
        ########################################RSI##################################################################
        #df = RSI(df,isFutureVal=0)
        df['RSI_14'] = ta.RSI(df['Close'],timeperiod = 14)
        ##########################################################################################################

        ##########################################################################################################
        #df = MACD(df,isFutureVal=0)
        macd,signal,_=ta.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        df['MACD'] = macd
        df['MACD_signal'] = signal

        ##########################################################################################################
        #df = MFI(df,isFutureVal=0)
        df['MFI_14'] = ta.MFI(df['High'], df['Low'], df['Close'], df['Volume'], timeperiod=14)
        
        ##############################################################################################################
        #df = Stochastic(df,isFutureVal=0)
        df['slow_K_1'], df['slow_D_1'] = ta.STOCH(df['High'], df['Low'], df['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
        ################################################################################################################
        
        
        
        
        
        
        df = Slope(df,1,Slope_1_min)
        df = Slope(df,3,Slope_3_5_min)
        df = Slope(df,5,Slope_3_5_min)
        df = df.replace(np.inf, np.nan)
        df = ShiftNadd(df,max_min,1)
        df = ShiftNadd(df,max_min,2)
        df = ShiftNadd(df,max_min,3)
        df = ShiftNadd(df,[ele+'_slope_1' for ele in Slope_1_min],1)
        df = ShiftNadd(df,[ele+'_slope_1' for ele in Slope_1_min],2)
        df = ShiftNadd(df,[ele+'_slope_1' for ele in Slope_1_min],3)
        #
        df=futurePrice(df,3)
        df=futurePrice(df,5)
        df=futurePrice(df,7)
        df=futurePrice(df,12)
        df=futurePrice(df,15)
        df['DAY_Kimchi_PriMean_5'] = df.apply(lambda x :np.nan if np.isnan(x['DAY_Kimchi_PriMean_5']) else round(x['DAY_Kimchi_PriMean_5']),axis=1)
        df['MaxFuture_510'] = df.apply(lambda x: np.nan if np.isnan(x['MaxFuture_5']) else ( 1 if x['MaxFuture_5']>0.1 else 0 )  ,axis=1)
        df['MaxFuture_515'] = df.apply(lambda x: np.nan if np.isnan(x['MaxFuture_5']) else ( 1 if x['MaxFuture_5']>0.15 else 0 )  ,axis=1)
        df['MaxFuture_520'] = df.apply(lambda x: np.nan if np.isnan(x['MaxFuture_5']) else ( 1 if x['MaxFuture_5']>0.2 else 0 )  ,axis=1)
        df['MaxFuture_5'] = df.apply(lambda x: np.nan if np.isnan(x['MaxFuture_5']) else ( 1 if x['MaxFuture_5']>0.3 else 0 )  ,axis=1)
        df['MaxFuture_310'] = df.apply(lambda x: np.nan if np.isnan(x['MaxFuture_3']) else ( 1 if x['MaxFuture_3']>0.1 else 0 )  ,axis=1)
        df['MaxFuture_315'] = df.apply(lambda x: np.nan if np.isnan(x['MaxFuture_3']) else ( 1 if x['MaxFuture_3']>0.15 else 0 )  ,axis=1)
        df['MaxFuture_320'] = df.apply(lambda x: np.nan if np.isnan(x['MaxFuture_3']) else ( 1 if x['MaxFuture_3']>0.2 else 0 )  ,axis=1)
        df['MaxFuture_3'] = df.apply(lambda x: np.nan if np.isnan(x['MaxFuture_3']) else ( 1 if x['MaxFuture_3']>0.3 else 0 )  ,axis=1)
        df['MaxFuture_7'] = df.apply(lambda x: np.nan if np.isnan(x['MaxFuture_7']) else ( 1 if x['MaxFuture_7']>0.4 else 0 )  ,axis=1)    
        df['MaxFuture_12'] = df.apply(lambda x: np.nan if np.isnan(x['MaxFuture_12']) else ( 1 if x['MaxFuture_12']>0.5 else 0 )  ,axis=1)   
        df['MaxFuture_15'] = df.apply(lambda x: np.nan if np.isnan(x['MaxFuture_15']) else ( 1 if x['MaxFuture_15']>0.6 else 0 )  ,axis=1) 
        
        df_arr_v1.append(df)
    globals()[cryptoName+'_1_Binance_'+str(year)+'_'+str(month)+'_df_arr'] = df_arr_v1

0.0
0.3333333333333333
0.6666666666666666


In [16]:
globals()[cryptoName+'_1_Binance_'+str(year)+'_'+str(month)+'_df_arr'][0].columns[0:20]

Index(['Open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 'quote_av',
       'trades', 'tb_base_av', 'tb_quote_av', 'Symbol', 'DAY_UTC_Day',
       'DAY_opening_price', 'DAY_Kimchi_Pri', 'DAY_Kimchi_PriMean_5',
       'DAY_Kimchi_Priunit_STD_5', 'DAY_Kimchi_PriMin_5',
       'DAY_Kimchi_PriMax_5', 'DAY_Kimchi_PriMean_20',
       'DAY_Kimchi_Priunit_STD_20'],
      dtype='object')

# Parameter-tunning with Optuna

In [17]:

# import numpy as np
# import optuna

# import lightgbm as lgb
# import sklearn.datasets
# import sklearn.metrics
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import f1_score

# # FYI: Objective functions can take additional arguments
# # (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
# def objective(trial):
#     #[310,315,320,3,7,12,15]
#     feature_= 15
#     year = 2021
#     month = 7
#     trainStartMonth=7
#     test_year=2021
#     test_month=10
    
#     df = pd.concat(globals()[cryptoName+'_1_Binance_'+str(year)+'_'+str(month)+'_df_arr'])

#     df_test = df[str(test_year)+'-'+str(test_month)]
#     df_train = df[:str(test_year)+'-'+str(test_month-1)]
    
#     df_train_= df_train[str(year)+'-'+str(trainStartMonth):]
    
#     train_x, train_y = trainTestConvert(df_train_,feature=feature_)
#     valid_x, valid_y = trainTestConvert(df_test,feature=feature_)
#     dtrain = lgb.Dataset(train_x, label=train_y)
#     #print(train_x)
#     #print(valid_x)
#     param = {
        

#         'objective': 'binary',
#         'boosting_type': 'gbdt',
#         'verbose': 100,
#         'metric': 'AUC',
#         'is_unbalance': True,
#         'boost_from_average': False,

#         "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
#         "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
#         "num_leaves": trial.suggest_int("num_leaves", 2, 1024),
#         "feature_fraction": trial.suggest_float("feature_fraction", 0.3, 1.0),
#         "bagging_fraction": trial.suggest_float("bagging_fraction", 0.3, 1.0),
#         "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
#         "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
#     }

#     gbm = lgb.train(param, dtrain)
#     preds = gbm.predict(valid_x)
#     #preds = [1 if ele >=0.6  else 0 for ele in preds]
# #     print(preds)
# #     print(valid_y)
#     #pred_labels = np.rint(preds)
#     score = roc_auc_score(valid_y, preds)
#     return score


# if __name__ == "__main__":
#     study = optuna.create_study(direction="maximize")
#     study.optimize(objective, n_trials=100)

#     print("Number of finished trials: {}".format(len(study.trials)))

#     print("Best trial:")
#     trial = study.best_trial

#     print("  Value: {}".format(trial.value))

#     print("  Params: ")
#     for key, value in trial.params.items():
#         print("    {}: {}".format(key, value))
#     del df_train_,X_train,X_test,y_train,y_test

# Train the model with the parameters from the previous step

In [18]:

import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
#Parameters from the optuna resutls.
params = {
        'task': 'train',
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': 100,
        'metric': 'auc',
        'is_unbalance': True,
        'boost_from_average': False,
    'lambda_l1': 1.2869878971560975e-06,
    'lambda_l2': 1.727941028953346e-08,
    'num_leaves': 9,
    'feature_fraction': 0.31389678481314626,
    'bagging_fraction': 0.5387311691309541,
    'bagging_freq': 1,
    'min_child_samples': 85
    }


for cryptoName in usdt_crypto_list:
    year = 2021
    month = 7
    df = pd.concat(globals()[cryptoName+'_1_Binance_'+str(year)+'_'+str(month)+'_df_arr'])
    test_year=2021
    test_month=10
    
    model_arr=[]
    
    df_test = df[str(test_year)+'-'+str(test_month)]
    df_train = df[:str(test_year)+'-'+str(test_month-1)]
    for idx,feature_ in enumerate([3]):
    
        score_arr=[]
        model_arr=[]
        for year in [2021]:
            if year == test_year:
                for trainStartMonth in range(7,8):
                    print(trainStartMonth)
                    df_train_= df_train[str(year)+'-'+str(trainStartMonth):]

                    X_train, y_train = trainTestConvert(df_train_,feature=feature_)
                    X_test, y_test = trainTestConvert(df_test,feature=feature_)
                    print(y_train)
                    print('#######################')
                    print(y_test)
                    print(X_train.shape,X_test.shape)
                    print(y_train.shape,y_test.shape)
                    model, score = fit_lgbm(
                    X_train, y_train,
                    X_test, y_test,
                    params
                    )
                    
                    score_arr.append((trainStartMonth,score))
                    model_arr.append(model)
                        
            else:
                for trainStartMonth in range(1,13):
                    print(trainStartMonth)
                    df_train_ = df_train[str(year)+'-'+str(trainStartMonth):]

                    X_train, y_train = trainTestConvert(df_train_,feature=feature_)
                    X_test, y_test = trainTestConvert(df_test,feature=feature_)
                    print(X_train.index)
                    print('#######################')
                    print(X_test.index)
                    
                    
                    model = lgb.train(params[idx], X_train)
                    preds = model.predict(X_test)
                    score = roc_auc_score(y_test, preds)                    
                    score_arr.append((year,trainStartMonth,score))
                    model_arr.append((year,trainStartMonth,model))
        globals()[cryptoName+'_1_Binance_'+str(year)+'_'+str(month)+'_score_arr'+str(feature_)] = score_arr
        globals()[cryptoName+'_1_Binance_'+str(year)+'_'+str(month)+'_model_arr'+str(feature_)] = model_arr

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.


7
time_index
2021-07-01 00:00:00    0.0
2021-07-01 00:01:00    0.0
2021-07-01 00:02:00    0.0
2021-07-01 00:03:00    0.0
2021-07-01 00:04:00    0.0
                      ... 
2021-09-30 23:55:00    0.0
2021-09-30 23:56:00    0.0
2021-09-30 23:57:00    0.0
2021-09-30 23:58:00    0.0
2021-09-30 23:59:00    1.0
Name: target, Length: 132084, dtype: float64
#######################
time_index
2021-10-01 00:00:00    1.0
2021-10-01 00:01:00    1.0
2021-10-01 00:02:00    1.0
2021-10-01 00:03:00    0.0
2021-10-01 00:04:00    0.0
                      ... 
2021-10-20 10:11:00    0.0
2021-10-20 10:12:00    0.0
2021-10-20 10:13:00    0.0
2021-10-20 10:14:00    0.0
2021-10-20 10:15:00    0.0
Name: target, Length: 27976, dtype: float64
(132084, 282) (27976, 282)
(132084,) (27976,)
[LightGBM] [Warning] lambda_l1 is set=1.2869878971560975e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.2869878971560975e-06
[LightGBM] [Warning] bagging_fraction is set=0.5387311691309541, subsample=1.0 wil

In [19]:
for cryptoName in usdt_crypto_list:
    for feature_ in [3]:
#     for feature_ in [310,315,320,3,510,515,520,5,7,12,15]:
        print(cryptoName,': ',feature_)
        print(globals()[cryptoName+'_1_Binance_'+str(year)+'_'+str(month)+'_score_arr'+str(feature_)])

ETH :  3
[(7, 0.5801426295359605)]


In [20]:
year=2021
month=7
import pickle
for cryptoName in usdt_crypto_list:
     for feature_ in [3]:
        print(cryptoName,': ',feature_)
        
        file = cryptoName+'_1_Binance_'+str(year)+'_'+str(month)+'_'+str(feature_)+'.pkl'
        print(file)
        pickle.dump(globals()[cryptoName+'_1_Binance_'+str(year)+'_'+str(month)+'_model_arr'+str(feature_)], open(file, 'wb'))
        print('Trained LGB model was saved!')

ETH :  3
ETH_1_Binance_2021_7_3.pkl
Trained LGB model was saved!


In [21]:
#from sklearn.externals import joblib
import joblib
for cryptoName in usdt_crypto_list:
    #for feature_ in [310,315,320,3,7,12,15]:
    for feature_ in [3]:
        
        print(cryptoName,': ',feature_)
    
        file = cryptoName+'_1_Binance_'+str(year)+'_'+str(month)+'_2_'+str(feature_)+'.pkl'

        joblib.dump(globals()[cryptoName+'_1_Binance_'+str(year)+'_'+str(month)+'_model_arr'+str(feature_)], file)



ETH :  3
